# Bike Station Analysis Project

Hubway, the government bike program, for the Boston metropolitan area is being analyzed in preparation for a city budget meeting.  Bike station and trip data from July 28, 2011 through October 1, 2012 comprise the set of data. 

In [2]:
# import libraries
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import pandas as pd
import datetime as dt

# import data sets and parse time/date columns
stations = pd.read_csv("ExData_stations.csv")
trips = pd.read_csv("ExData_trips.csv", parse_dates=[3,5])
capacities = pd.read_csv("stationcapacity.csv", parse_dates=[2])

## Most and Least Used Stations

In [3]:
# creating aggregated values for start and end stations
stat = trips.groupby("start_station").aggregate(len)
stat2 = trips.groupby("end_station").aggregate(len)
# creating variables to call the aggregated station data by trip id
stat_count1 = stat["id"]
stat_count2 = stat2["id"]
stat_count3 = stat["id"]+stat2["id"]

print ("The most/least used starting station\n", stat_count1.sort_values())
print ("The most/least used ending station\n", stat_count2.sort_values())
print ("How much each station was used (combined values for start/end)\n", stat_count3.sort_values())

The most/least used starting station
 start_station
92.0      2
96.0      6
94.0      6
87.0      8
95.0     10
       ... 
16.0    535
42.0    597
38.0    768
36.0    778
22.0    978
Name: id, Length: 94, dtype: int64
The most/least used ending station
 end_station
92.0      2
93.0      3
95.0      6
96.0      7
77.0      8
       ... 
33.0    509
42.0    612
36.0    793
38.0    824
22.0    946
Name: id, Length: 95, dtype: int64
How much each station was used (combined values for start/end)
 92.0       4.0
96.0      13.0
95.0      16.0
87.0      19.0
94.0      20.0
         ...  
42.0    1209.0
36.0    1571.0
38.0    1592.0
22.0    1924.0
93.0       NaN
Name: id, Length: 95, dtype: float64


The most used start station was 22 which was used 978 times.  The least used start station was 92 with 2 trips.
The most used end station was 22 with 946 trips.  The least used end station was 92 with 2 trips.
The most used station overall is 22 which was used 1,924 times.  The least used station overall was 92 which had a total of 4 trips.  
Station 93 is an outlier with no trips.

## Home-based vs Commuter Stations

In [4]:
# creating a new column called station_type which indicates whether a trip starts and ends in the same place (home_based)
# or in different locations (commuter).
trips["station_type"] = np.where((trips["start_station"] == trips["end_station"])
                     , 'home_based', 'commuter')

trips.sort_values("start_station")
print(trips.head())

# aggregate number of stations by home_based or commuter
stat3 = trips.groupby("station_type").aggregate(len)
stat_count4 = stat3["id"]
stat_count4

    id  status  duration                 start_date  start_station  \
0  119  Closed       394  2011-07-28 13:16:00-04:00           10.0   
1  174  Closed       386  2011-07-28 15:55:00-04:00           24.0   
2  218  Closed       589  2011-07-28 16:56:00-04:00           22.0   
3  238  Closed      1425  2011-07-28 17:16:00-04:00            9.0   
4  276  Closed       465  2011-07-28 17:52:00-04:00           44.0   

                    end_date  end_station bike_nr subscription_type zip_code  \
0  2011-07-28 13:23:00-04:00         32.0  B00189        Registered    02492   
1  2011-07-28 16:01:00-04:00         48.0  B00082        Registered    02113   
2  2011-07-28 17:05:00-04:00         49.0  B00137        Registered    01890   
3  2011-07-28 17:40:00-04:00          9.0  B00329            Casual      NaN   
4  2011-07-28 18:00:00-04:00         22.0  B00136            Casual      NaN   

   birth_date gender station_type  
0      1975.0   Male     commuter  
1      1986.0   Male     c

station_type
commuter      18626
home_based     1374
Name: id, dtype: int64

## Sending Commuter Stations with Primarily Morning, Evening, or Balanced Net Outflow  

In [30]:
# create a variable container for commuter stations only
sending = trips[trips["station_type"] == "commuter"]

# separating time data into a separate hour column
s = sending.start_date.apply(lambda x: dt.datetime(year=x.year, month=x.month, day=x.day, hour=x.hour))
sending["Hour"] = s.dt.hour

# creating and calling a variable for aggregating count of trips by hour
time_use = sending.groupby("Hour").aggregate(len)
send = time_use["id"]
send

<ipython-input-30-50c1600ed19d>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sending["Hour"] = s.dt.hour


Hour
0      190
1      162
2      158
3       54
4       20
5       92
6      355
7      872
8     1506
9      860
10     713
11     855
12    1083
13    1184
14    1070
15    1185
16    1568
17    2067
18    1462
19    1073
20     705
21     607
22     452
23     333
Name: id, dtype: int64

There is much heavier use in the second half of the day.  8am is a noteable outlier in the morning as people are heading to work.

## Bikes Taken and Not Returned (with Trip Data)  

In [6]:
# looking for trips with no end station (null values in the end_station column)
missing = trips[trips["end_station"] == pd.NA]
missing

,id,status,duration,start_date,start_station,end_date,end_station,bike_nr,subscription_type,zip_code,birth_date,gender,station_type


In [7]:
# looking for trips with no end_date (null values in the end_date column)
missing2 = trips[trips["end_date"] == pd.NA]
missing2

,id,status,duration,start_date,start_station,end_date,end_station,bike_nr,subscription_type,zip_code,birth_date,gender,station_type


In [8]:
# looking for trips without a closed status (potential unfinished trips with bikes still out)
missing3 = trips[trips["status"] != "Closed"]
missing3

,id,status,duration,start_date,start_station,end_date,end_station,bike_nr,subscription_type,zip_code,birth_date,gender,station_type


All bikes had a closed status and an end_station and end_date, indicating that there are no missing bikes

## Popularity of use by day of the week

In [9]:
# create day of week variable to convert datetime day numbers to names
dayOfWeek={0:'Monday', 1:'Tuesday', 2:'Wednesday', 3:'Thursday', 4:'Friday', 5:'Saturday', 6:'Sunday'}
# separating time data into a separate day of the week column
s = trips.start_date.apply(lambda x: dt.datetime(year=x.year, month=x.month, day=x.day))
trips["Day"] = s.dt.dayofweek.map(dayOfWeek)

# aggregating trips by Day of the Week
day_count = trips.groupby("Day")["id"].aggregate(len)
day_count

Day
Friday       3062
Monday       2892
Saturday     2769
Sunday       2743
Thursday     2891
Tuesday      2780
Wednesday    2863
Name: id, dtype: int64

Friday is the most popular day of the week to take a bike out and Sunday is the least popular day.  Weekdays are more popular than weekends.

## Capacity considerations

### Average station capacity

In [42]:
# Find the average bike capacity of each station
cap = capacities.groupby("station_id").mean()
cap.sort_values("capacity")

,id,capacity
station_id,,
1,4123.261905,0.000000
28,5148.706667,0.000000
35,10121.758112,10.471976
32,10097.476471,10.550000
47,10291.888554,10.641566
...,...,...
21,10066.064516,21.445748
70,18789.945946,22.932432
100,21705.272727,24.818182


As alluded to above, station 22 runs the most overall trips, but has the second largest average capacity behind station 38.  Station 38 runs the second largest amount of trips.  There are also unused stations not accounted for in station database (new?).

### Capacity by station and day of the week

In [63]:
# separating time data into a separate day of the week column for capacity data
daycap = capacities.day.apply(lambda x: dt.datetime(year=x.year, month=x.month, day=x.day))
capacities["day_of_week"] = daycap.dt.dayofweek.map(dayOfWeek)

# finding a mean capacity by day of the week for each station
grouped_multiple = capacities.groupby(['station_id', 'day_of_week']).agg({'capacity': ['mean']})
print(grouped_multiple)

                       capacity
                           mean
station_id day_of_week         
1          Friday           0.0
           Monday           0.0
           Saturday         0.0
           Sunday           0.0
           Thursday         0.0
...                         ...
105        Friday          19.0
           Monday          19.0
           Saturday        19.0
           Sunday          19.0
           Tuesday         19.0

[718 rows x 1 columns]


This code will allow for a slice of any particular station to see if the day of the week affects capacity for a particular station.

## Demographic statistics for Registered Users

### Gender Usage

In [12]:
# aggregating trips by gender
gender_use = trips.groupby("gender").aggregate(len)
gender_use.id

gender
Female    3257
Male      9677
Name: id, dtype: int64

Males are almost 3 times as likely as females to use the bike service.

### Usage by Age

In [13]:
# aggregating trips by birth year
age_use = trips.groupby("birth_date").aggregate(len)
age_use.id

birth_date
1932.0      1
1938.0      4
1939.0      3
1940.0      2
1942.0      8
1943.0      1
1944.0      4
1945.0     17
1946.0     30
1947.0     33
1948.0     28
1949.0     43
1950.0     64
1951.0     56
1952.0    101
1953.0    124
1954.0     64
1955.0    113
1956.0     98
1957.0    184
1958.0    134
1959.0    194
1960.0    162
1961.0    182
1962.0    245
1963.0    213
1964.0    125
1965.0    211
1966.0    229
1967.0    224
1968.0    179
1969.0    202
1970.0    302
1971.0    293
1972.0    201
1973.0    306
1974.0    311
1975.0    288
1976.0    328
1977.0    407
1978.0    317
1979.0    370
1980.0    565
1981.0    520
1982.0    566
1983.0    727
1984.0    636
1985.0    600
1986.0    663
1987.0    559
1988.0    436
1989.0    415
1990.0    234
1991.0    233
1992.0    151
1993.0     90
1994.0     40
1995.0      5
Name: id, dtype: int64

Those born in 1983 are most likely to use the service with birthdates in the 1980's being the most popular age range of usage.

### Usage by Zip Code

In [14]:
# aggregating trips by user's zip code
location_use = trips.groupby("zip_code").aggregate(len)
location_use.id.sort_values(ascending=False)

zip_code
02118    1918
02116    1167
02215    1161
02115     775
02113     548
         ... 
03301       1
01606       1
11797       1
01590       1
02126       1
Name: id, Length: 301, dtype: int64